## The issue with perfect phylogenies

Previously, I wrote [a blog post](https://genomejigsaw.wordpress.com/2015/09/09/building-phylogenetic-trees-with-binary-traits/), exploring an algorithm for building phylogenetic trees from binary traits. While the algorithm works well if you happen to have a clean matrix that just-so happes to form a perfect phylogeny, if you experiment with different matrix permutations of 1s and 0s, you'll quickly find that matrices which create perfect phylogenies are scarce. In fact, if there are any pairs of columns \[C1,C2\] have the values \[0,1\], \[1,0\] and \[1,1\], then a perfect phylogeny cannot be built. Imperfect data sources and noisy data can contribute to these difficulties of building a valid phylogeny. To allow for some slack for running the phylogeny-finding algorithm, one option is to use the imperfect phylogeny algorithm, an algorithm implemented in near-linear time, described in a [paper by Pe'er et al.](http://epubs.siam.org/doi/abs/10.1137/S0097539702406510) (unfortunately, the paper is pay-walled). I will explain and implement this algorithm in code in this post. If you're after the mathematical proofs, refer to the original paper. 

In modelling phylogenetic trees from binary data, it is unlikely that you'll have absolute certainty in observing data values. In my early experiments of my PhD project, I was utilising the [Gusfield algorithm](http://carolineuhler.com/paperCS.pdf) for [structural variation](http://www.ncbi.nlm.nih.gov/dbvar/content/overview/) data in multiple samples of the same cancer patient. In my case, observing the same structural variation across several tumour samples of the same patient gave me high confidence that the variation existed. However, I was much less confident when I did _not_ see variant in one patient sample, when it was clearly in others. Thus, my '1's were certain, but my '0's were uncertain. In my case, there were several reasons why we might not see a variant in a particular sample (maybe the [depth of coverage](https://en.wikipedia.org/wiki/Deep_sequencing) was too low, or the sample had a lot of normal tissue contamination). In this case, I could mark the rows of my low-quality samples with 'incomplete' or unknown values where SVs were not found. This strategy allowed me to find valid phylogenies where previously they failed to be built. 

## The incomplete phylogeny problem

In our feature matrix, we now allow a third feature type - unknown or incomplete, denoted by '?'. Hence our values can be 0, 1 or ? for any cell. The incomplete phylogeny algorithm will then infer the missing values by determining whether the sample belongs to the 'clade' associated with the feature - the clade being the evolutionary group with a common ancestor, that the sample/species evolved from. 

The algorithm will output a perfect phylogeny tree _T_ (if present), as well as the corresponding completed matrix _M_. The following steps are involved:

1. Remove any columns in our matrix _M_ with no 0s or all 0s.
2. Construct a graph where 1s represent edges between features (columns) and samples (rows).
3. Get each set of connections where there are >1 connections in the graph (call these _K_), for each _K_ do:

    i. get all the samples in _K_, call this _S'_
    
    ii. for this set of _S'_, get all the features where there are no 0 entries, call this _U_
    
    iii. if there are no elements in _U_, stop
    
    iv. otherwise remove _U_ from the graph, and add _S'_ to our tree _T_

To implement the algoprithm, Let's work from the following matrix:

|    | C1 | C2 | C3 | C4 | C5 |
|----|----|----|----|----|----|
| S1 | 1  | 1  | 0  | 0  | ?  |
| S2 | 0  | ?  | 1  | 0  | ?  |
| S3 | 1  | ?  | 0  | 0  | 1  |
| S4 | 0  | 0  | 1  | 1  | ?  |
| S5 | 0  | 1  | 0  | 0  | ?  |


There are no columns with all 0s or no 0s, so we do not need to discard any columns. We will now transform this matrix into a graph, creating connections between the nodes, $C$s and $S$s. Where a $C$ and $S$ share a value of '1', we draw an edge between them. To do this, we first code up an graph implementation, with nodes and vertices. One approach can be found [here](http://interactivepython.org/runestone/static/cs22-lbcc/Graphs/graphintro.html) - the base graph code, as well as the _M_ Graph implementation are in the mgraph.py file in this repository.

In [1]:
from mgraph import MGraph

We now need some helper methods to help build our graph. The _build_\__graph_ function below creates a node for each of our $S$ and $C$ values, and connects them with an edge if the corresponding $(S_i,C_j$) cell contains a 1. We then define a function _get_\__edge_\__pairs_ to get all the pairwise connections between the nodes. 

We now build our arrays for our $S$s and $C$s, as well as or _M_ matrix. Let's use -1 to represent the 'incomplete' state of our matrix. 

In [2]:
from mgraph import MGraph
import numpy as np

s = np.array(['s1','s2','s3','s4','s5'])
c = np.array(['c1','c2','c3','c4','c5'])
m = np.array([[ 1,  1,  0,  0, -1],
              [ 0, -1,  1,  0, -1],
              [ 1, -1,  0,  0,  1],
              [ 0,  0,  1,  1, -1],
              [ 0,  1,  0,  0, -1]])

# create working matrix M and column C vector copies
m3 = m.copy()
c3 = c.copy()

# remove any columns where there are no 0 entries
mi = np.empty(0,dtype='int')
ncol = len(m3[0])
idxs_to_delete = [i for i in range(ncol) if not np.any(m3[:,i:i+1]==0)]
m3 = np.delete(m3,idxs_to_delete,axis=1)
c3 = np.delete(c3,idxs_to_delete)

m_graph = MGraph()
m_graph.build_graph(m3,s,c3)
m_pairs = m_graph.get_edge_pairs()

print('Nodes in our graph')
print(m_graph.getNodes())

print('\nConnections between out nodes')
print(m_graph.get_edge_pairs())

Nodes in our graph
['s3', 's2', 's1', 's5', 's4', 'c3', 'c2', 'c1', 'c4']

Connections between out nodes
[['s3', 'c1'], ['s2', 'c3'], ['s1', 'c1'], ['s1', 'c2'], ['s5', 'c2'], ['s4', 'c4'], ['s4', 'c3']]


The variable m_cons above displays the graph connections between our $S$s and $C$s. We now get the first set of connections in the graph, and determine the K vector (which is the union of all $S$s and $C$s involved in the graph connection). 

In [4]:
def get_k(k,q,m_graph):
    '''
    return the first K vector with E[K] >= 1
    (the K vector contains all elements of a chain of connected 
    vertices with more than just one connection between them)
    '''
    if not q:
        return k
    else:
        q1 = q.pop()
        k.add(q1) 
        pairs = m_graph.get_pairs_containing(q1)
        pairs = set([node for pair in pairs for node in pair]) #flatten set of elements
        for node in pairs:
            if node not in k:
                q.add(node)
        return get_k(k,q,m_graph)

q = set(m_pairs[0]) #pick the first elements as k
print(q)
k = get_k(set(),q,m_graph)
print(k)

set(['s3', 'c1'])
set(['s3', 'c2', 'c1', 's1', 's5'])


Based on the $S$s returned in the connections, we now return the slice of the $M_3$ array, containing the $S$s contained in set $K$. 

In [55]:
s_prime = set(s).intersection(k)
s_indexes = np.array([np.where(s_i==s)[0][0] for s_i in s_prime])
m_tmp = m3.copy()[s_indexes]
m_tmp

array([[ 1, -1,  0,  0],
       [ 1,  1,  0,  0],
       [ 0,  1,  0,  0]])

Now we return our $U$ vector, which is defined as any $C$s where there are no $0$s. 

In [56]:
u = []
c_in_k = set(c).intersection(k)
for c_i in c_in_k:
    c_index = np.where(c_i==c3)[0]
    m_col = m_tmp[:,c_index:c_index+1]
    if np.all(m_col!=0):
        cm = np.where(c_i==c3)[0]
        u.append(c_i)
print(u)

['c2']


Now any elements that are in the character columns contained in $U$, *and* in the sample rows contained in $S'$ (i.e. they belong to the same clade), will have any incomplete fields inferred as present. 

In [103]:
m_new = m3.copy()

c_indexes = [np.where(c3==x)[0][0] for x in u]
for c_idx in c_indexes:
    print(c_idx)
    m_col = m_new[s_indexes,c_idx:c_idx+1]
    if np.any(m_col==-1):
        m_col[m_col==-1] = 1
        m_new[s_indexes,c_idx:c_idx+1] = m_col
        
print("Incomplete matrix prior to algorithm step")
print(m3)
print("\nMatrix after first round of algorithm")
print(m_new)

1
Incomplete matrix prior to algorithm step
[[ 1  1  0  0]
 [ 0 -1  1  0]
 [ 1 -1  0  0]
 [ 0  0  1  1]
 [ 0  1  0  0]]

Matrix after first round of algorithm
[[ 1  1  0  0]
 [ 0 -1  1  0]
 [ 1  1  0  0]
 [ 0  0  1  1]
 [ 0  1  0  0]]


Hence in the above example, the matrix $M_3$ cell corresponding to $S3$ and $C2$ has been inferred to be present (i.e. 1). 

In [12]:
t = [0,set(s)] #initialiase a tree
t.append(s_prime)
print(t)

[0, set(['s3', 's2', 's1', 's5', 's4']), set(['s3', 's1', 's5'])]


In [ ]:
m_graph.remove_nodes_in(u)
m_pairs = m_graph.get_edge_pairs()